# TP Représentation symbolique du langage naturel et inférence

In [1]:
# First import what will be needed
import nltk

## Analyse syntaxique

L'analyse syntaxique du langage naturel est complexe, car le langage naturel ne possède pas la rigueur des langages formels.
La polysémie est une des caractéristiques qui rend le langage naturel difficile à analyser, mais qui en fait aussi la richesse. La polysémie se produit :
- au niveau des mots : `glace` est un nom et un verbe, `par` est un nom (golf) et un adverbe,
- au niveau des phrases : `la petite brise la glace` peut se comprendre de plusieurs manières
- au niveau de textes entiers : l'histoire regorge de paraboles comme les fables de La Fontaine destinées à critiquer tel pouvoir sans le nommer explicitement. Elle repose sur des grammaires formelles du type hors-contexte auxquelles on peut adjoindre différents mécanismes.

Néammoins, l'analyse syntaxique du langage naturel repose sur l'utilisation de grammaires hors-contexte que l'on augmentera par divers mécanismes dont certains sont illustrés ci-dessous.

Tout d'abord, examinons comment NLTK permet d'analyser un énoncé selon une grammaire :

In [2]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP
V -> 'catches' | 'sees'
NP -> 'Alice' | 'Bob' | Det N
Det -> 'a' | 'an' | 'the'
N -> 'cat' | 'dog' | 'mouse' | 'saucepan'
""")

In [3]:
parser = nltk.RecursiveDescentParser(grammar)
sentence = "Alice catches the mouse".split()
for tree in parser.parse(sentence):
    print(tree)
    # tree.draw() # Attention, ouvre une fenêtre externe


(S (NP Alice) (VP (V catches) (NP (Det the) (N mouse))))


![Arbre d'analyse de la phrase *Alice catches the mouse*](images/img_2022-09-05-21-36-53.png)

L'analyse repose sur un algorithme récursif descendant dans ce cas.
La bibliothèque NLTK fournit une large variété d'algorithmes d'analyse syntaxique avec diverses caractéristiques.

Examinons le cas un peu particulier de la phrase :

*la petite brise la glace*

Nous écrivons un fragment de grammaire pour le français :
- GN désigne un groupe nominal
- GV désigne un groupe verbal
- Un groupe nominal est composé d'un article, éventuellement d'un adjectif, puis d'un nom
- Un groupe verbal est composé d'un verbe suivi d'un complément, ou bien d'un complément sous forme de pronom suivi d'un verbe

In [4]:
grammar = nltk.CFG.fromstring("""
S -> GN GV
GV -> V GN | Pronom V
GN -> Art N | Art Adj N
Art -> 'le' | 'la'
V -> 'brise' | 'glace'
N -> 'brise' | 'glace' | 'petit' | 'petite'
Adj -> 'petit' | 'petite'
Pronom -> 'la' | 'lui'
""")

puis nous analysons la phrase :

In [5]:
parser = nltk.RecursiveDescentParser(grammar)
sentence = "la petite brise la glace".split()
for tree in parser.parse(sentence):
    print(tree)
    tree.draw()

(S (GN (Art la) (N petite)) (GV (V brise) (GN (Art la) (N glace))))


Ici, on obtient 2 arbres d'analyses car il y a ambiguité :

![](images/img_2022-09-05-21-45-34.png)

![](images/img_2022-09-05-21-47-30.png)

Vous pouvez écrire vos propres grammaires dans un fichier séparé, puis les charger avec la commande :

In [5]:
grammar1 = nltk.data.load('file:grammar/mygrammar.cfg')
parser = nltk.RecursiveDescentParser(grammar1)
sentence = "the angry bear chased the frightened little squirrel".split()
for tree in parser.parse(sentence):
    print(tree)
    tree.draw()

(S
  (NP (Det the) (Nom (Adj angry) (Nom (N bear))))
  (VP
    (V chased)
    (NP
      (Det the)
      (Nom (Adj frightened) (Nom (Adj little) (Nom (N squirrel)))))))


KeyboardInterrupt: 

: 

![](images/img_2022-09-05-22-01-26.png)



## Accords et traits

On remarquera que pour l'instant, nos exemples sont capables d'analyser des énoncés qui ne sont pas syntaxiquement corrects pour le français.

In [5]:
grammar = nltk.CFG.fromstring("""
S -> GN GV
GV -> V GN | Pronom V
GN -> Art N | Art Adj N
Art -> 'le' | 'la'
V -> 'brise' | 'glace'
N -> 'brise' | 'glace' | 'petit' | 'petite'
Adj -> 'petit' | 'petite'
Pronom -> 'le' | 'la' | 'lui'
""")
parser = nltk.RecursiveDescentParser(grammar)
sentence = "le petit brise le glace".split()
for tree in parser.parse(sentence):
    print(tree)


(S (GN (Art le) (N petit)) (GV (V brise) (GN (Art le) (N glace))))
(S (GN (Art le) (Adj petit) (N brise)) (GV (Pronom le) (V glace)))


Aucun accord de genre n'est respecté et il en serait de même du nombre.
L'introduction de ce type de contrainte dans une grammaire CFG ne peut se faire qu'au prix de la duplication des non-terminaux : il faudrait introduire un adjectif masculin, un adjectif féminin, un nom masculin, un nom féminin, etc. et encore multiplier par deux pour les versions singulier et pluriel ... la grammaire explose et devient impossible à maintenir.
Pourtant, malgré leur multiplication, les règles ont la même structure. 

On a donc introduit un mécanisme permettant d'éviter la multiplication des non-terminaux et de propager des contraintes au travers de l'arbre d'analyse.
Les non-terminaux seront décorés par un ensemble de traits dont les valeurs sont des termes formels.
Les valeurs des traits dans une règle de production devront être en accord.
Les traits peuvent avoir une variable pour valeur, dans ce cas, la valeur remontée pour ce trait est obtenue par unification entre les valeurs de ce trait dans la règle.
Le principe est illustré ci-dessous avec deux traits GENRE et NOMBRE :

In [6]:
grammar = nltk.grammar.FeatureGrammar.fromstring("""
S -> GN[NOMBRE=?n] GV[NOMBRE=?n]
GV[NOMBRE=?n] -> V[NOMBRE=?n] GN | Pronom V[NOMBRE=?n] | V[NOMBRE=?n]
GN[NOMBRE=?n,GENRE=?g] -> Art[NOMBRE=?n,GENRE=?g] N[NOMBRE=?n,GENRE=?g] | Art[NOMBRE=?n,GENRE=?g] Adj[NOMBRE=?n,GENRE=?g] N[NOMBRE=?n,GENRE=?g]
Art[NOMBRE=sg,GENRE=m] -> 'le'
Art[NOMBRE=sg,GENRE=f] -> 'la'
Art[NOMBRE=pl] -> 'les'
V[NOMBRE=sg] -> 'brise' | 'glace'
V[NOMBRE=pl] -> 'brisent' | 'glacent'
N[NOMBRE=sg,GENRE=f] -> 'brise' | 'glace' | 'petite'
N[NOMBRE=pl,GENRE=f] -> 'brises' | 'glaces' | 'petites'
N[NOMBRE=sg,GENRE=m] -> 'petit' 
N[NOMBRE=pl,GENRE=m] -> 'petits' 
Adj[NOMBRE=sg,GENRE=m] -> 'petit'
Adj[NOMBRE=sg,GENRE=f] -> 'petite'
Adj[NOMBRE=pl,GENRE=m] -> 'petits'
Adj[NOMBRE=pl,GENRE=f] -> 'petites'
Pronom[NOMBRE=sg,GENRE=m] -> 'le'| 'lui'
Pronom[NOMBRE=sg,GENRE=f] -> 'la'
Pronom[NOMBRE=pl] -> 'les'
""")
parser = nltk.FeatureEarleyChartParser(grammar)
sentences = [ 'le petit brise la glace', 'les petits brisent la glace', 'le petite brise le glace', 'la petite brisent le glace', 'la petite brise le glace']
for sentence in sentences:
    for tree in parser.parse(sentence.split()):
        print(f'{sentence} -> {tree}')


le petit brise la glace -> (S[]
  (GN[GENRE='m', NOMBRE='sg']
    (Art[GENRE='m', NOMBRE='sg'] le)
    (N[GENRE='m', NOMBRE='sg'] petit))
  (GV[NOMBRE='sg']
    (V[NOMBRE='sg'] brise)
    (GN[GENRE='f', NOMBRE='sg']
      (Art[GENRE='f', NOMBRE='sg'] la)
      (N[GENRE='f', NOMBRE='sg'] glace))))
les petits brisent la glace -> (S[]
  (GN[GENRE='m', NOMBRE='pl']
    (Art[NOMBRE='pl'] les)
    (N[GENRE='m', NOMBRE='pl'] petits))
  (GV[NOMBRE='pl']
    (V[NOMBRE='pl'] brisent)
    (GN[GENRE='f', NOMBRE='sg']
      (Art[GENRE='f', NOMBRE='sg'] la)
      (N[GENRE='f', NOMBRE='sg'] glace))))
la petite brise le glace -> (S[]
  (GN[GENRE='f', NOMBRE='sg']
    (Art[GENRE='f', NOMBRE='sg'] la)
    (Adj[GENRE='f', NOMBRE='sg'] petite)
    (N[GENRE='f', NOMBRE='sg'] brise))
  (GV[NOMBRE='sg']
    (Pronom[GENRE='m', NOMBRE='sg'] le)
    (V[NOMBRE='sg'] glace)))


On voit ici que plusieurs énoncés qui ne sont pas accordés correctement ne sont pas analysés.
Le mécanisme de traits (NOMBRE, GENRE, etc.) est extrêmement puissant.
Vous trouverez plus d'informations et d'exemples en consultant ce [chapitre](https://www.nltk.org/book/ch08.html) sur le site de la bibliothèque NLTK.

Nous allons utiliser ce mécanisme pour remonter une valeur sémantique de l'énoncé : cette valeur sera obtenue par composition de la sémantique assignée aux composants individuels de l'énoncé.